In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from modules.tfidf_custom import TfIdfCustom
from modules.tfidf import TfIdf
from modules.word_stats import WordStats
from modules.similarity import Similarity
from models.knnreg import KnnReg
from modules.data import fetch_fresh, load_pickle, save_pickle
from modules.stats import score_distribution_plot, words_count_plot, tfidf_custom_score_plot, topic_similarity_score_plot
import pandas as pd
# from scipy.stats import binned_statistic
import sys
import argparse
pickle_df = load_pickle("okt-11-1499c.p")
display(pickle_df)

,body,subreddit,submission,tfidf_score,tfidf_custom_score,length,sentences,common_word,words_count,stop_words_count,bad_words_count,bad_words,letter_count,emoticons,aspectual_class,reichenbach_tense,topic_similarity,label
0,*What if Jod was one of us?*,askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,753
1,A large chunk of my taking the lord's name in ...,askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,41889
2,Do you want a Holy War? Because that's how you...,askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,26323
3,"He would also say, “my name isn’t God” then he...",askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,6585
4,"We’d be like, “No way!” and he’d be like, “Yah...",askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,20301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,"As was explained elsewhere, the basic answer i...",explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1
1495,Teeth don' t just get holes.\n\nThey get littl...,explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1
1496,Bone is a living tissue with direct blood supp...,explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1
1497,Teeth aren’t bones. They are derived from ecto...,explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1


In [2]:
pd.options.display.max_rows = 16
pd.options.display.max_columns = 20
%matplotlib inline
mpl.rcParams["figure.figsize"] = 15,8

In [3]:
df = pickle_df.copy()

In [4]:

similarity = Similarity()
similarity.score(df) # "topic_similarity"

word_stats = WordStats()
df = word_stats.score(df) # "words_count", "stop_words_count", "bad_words_count", "bad_words"

tfidf_custom = TfIdfCustom()
df = tfidf_custom.score(df) # "tfidf_custom_score"

In [33]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
stop_words = set(stopwords.words("english"))


class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', '[removed]', '>', '*', '_']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))

submissions = set(df.get("submission"))

for submission in submissions:
    submission_df = df[df.submission == submission]
    # use all stop words (and ignore_tokens) with the tokenizer interfaced for sklearn above
    vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer)
    doc_vectors = vectorizer.fit_transform([submission] + submission_df.get("body").values.tolist())
    tfidf_scores = []
    for i in range(doc_vectors.shape[0]):
        arr = doc_vectors[i].toarray()
        tfidf_scores.append(
            sum(arr[0]) / len(np.nonzero(arr[0]))
        )
    submission_df["tfidf_score"] = tfidf_scores[1:] # the first one is the submission itself
    df.update(submission_df)

idx = 1000
display(df[["body", "submission", "topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words", "label"]][idx:idx+12])

# display(df[["body", "submission", "topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words", "label"]][df["body"] == "[removed]"][idx:idx+12])

# place tf-idf values in a pandas data frame
#df_tfidf = pd.DataFrame(first_vector.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
#df_tfidf.sort_values(by=["tfidf"],ascending=False)
#
#df_tfidf[df_tfidf["tfidf"]!=0]




,body,submission,topic_similarity,tfidf_score,words_count,stop_words_count,bad_words_count,bad_words,label
1000,Same day the Cornell study dropped about him b...,President Donald Trump says he has tested posi...,0.000000,3.291424,12.0,7.0,0.0,,1811.0
1001,"May he receive all the kindness, love, sympath...",President Donald Trump says he has tested posi...,0.000000,3.450360,17.0,14.0,0.0,,429.0
1002,Regardless of your politics. Wear a goddamn mask.,President Donald Trump says he has tested posi...,0.000000,2.231736,7.0,3.0,0.0,,12308.0
1003,"“This has to be the best case of covid, ever. ...",President Donald Trump says he has tested posi...,0.042867,3.342683,19.0,7.0,0.0,,9298.0
1004,This could be the most upvoted Reddit post in ...,President Donald Trump says he has tested posi...,0.000000,1.998194,7.0,4.0,0.0,,1498.0
1005,2020 coming in with a plot twist,President Donald Trump says he has tested posi...,0.000000,1.994672,4.0,3.0,0.0,,34176.0
1006,Trump. Bolsanoro. Johnson. All three have test...,President Donald Trump says he has tested posi...,0.339098,2.399240,10.0,2.0,0.0,,577.0
1007,I want Dr. Fauci to confirm Trump’s sickness. ...,President Donald Trump says he has tested posi...,0.025896,5.792889,51.0,24.0,0.0,,5800.0
1008,Somebody call /r/leopardsatemyface,President Donald Trump says he has tested posi...,0.000000,1.727949,3.0,0.0,0.0,,6444.0
1009,Masks work - please continue to wear them to s...,President Donald Trump says he has tested posi...,0.000000,3.412545,14.0,8.0,0.0,,1050.0
